In [1]:
# To find values within a column...
#df.loc[df['column_name'] == some_value]
#x = dfs[0][1]
#x.loc[x["ncli"] == 8151]

In [1]:
import os
import re
import math
import pandas as pd
import numpy as np

import plotly
import plotly.plotly as py
from plotly.graph_objs import *

plotly.tools.set_credentials_file(username='theano', api_key='hkYRYxjyM0aAGA5nmGq1')

In [2]:
import matplotlib.pyplot as plt

import matplotlib
matplotlib.style.use('ggplot')

In [3]:
home = os.getcwd()
print(home)
sorted([x for x in os.listdir(home)])
collectl_stats = ['Date', '[CPU]User%', '[CPU]Nice%', '[CPU]Sys%', '[CPU]Wait%',
                               '[CPU]Irq%', '[CPU]Soft%', '[CPU]Steal%', '[CPU]Idle%',
                               '[CPU]Totl%', '[CPU]Intrpt/sec', '[CPU]Ctx/sec', '[CPU]Proc/sec',
                               '[CPU]ProcQue', '[CPU]ProcRun', '[CPU]L-Avg1', '[CPU]L-Avg5',
                               '[CPU]L-Avg15', '[CPU]RunTot', '[CPU]BlkTot']
loaded_files = []


/Users/theano/mountpoint/tsch/mininetpipe/results/2017-12-18-18-44


In [4]:
def load_datfile(fpath):
    # Returns a dataframe
    df = pd.read_csv(fpath, 
                     index_col=False,
                     names=["ncli","throughput"])
    return df


def load_collectl(fname):
    df = pd.read_csv(fname,
                     delim_whitespace=True,
                     comment="#",
                     names=collectl_stats)
    return df

def load_data_and_stats(basename):
    pds = []
    prog = re.compile("%s_\w+\.dat$" % basename)
    
    for fname in [x for x in os.listdir(".") if prog.match(x)]:
        df_tput = load_datfile(os.path.join(home, fname))
        
        print("Loading file %s" % fname)
        collectl_fname = [x for x in os.listdir(".") if "%s-server-0" % fname in x]
        if len(collectl_fname) > 0:
            df_stats = load_collectl(collectl_fname[0])
            newdf = pd.concat([df_tput.sum(), df_stats.mean()])
        else:
            newdf = df_tput.sum()
        
        nclients = re.search("(?<=_c)\d+", fname)
        try:
            nclients = int(nclients.group(0))
            if newdf["ncli"] != nclients:
                print("skipping df with %d clients...\n" % newdf["ncli"])
                continue
        except:
                print("exception")
                continue
            
        expduration = int(re.search("(?<=_u)\d+", fname).group(0))
        
        newdf["fname"] = fname
        newdf["expduration"] = expduration
        newdf["numa"] = int(fname.split("numa")[0])        
        pds.append(newdf)
        
    return (basename, (pd.concat(pds, axis=1)
                        .transpose()).apply(pd.to_numeric, errors='coerce'))

def load_data_no_sum(basename):
    pds = []
    prog = re.compile("%s_\w+\.dat$" % basename)
    
    for fname in [x for x in os.listdir(".") if prog.match(x)]:
        df_tput = load_datfile(os.path.join(home, fname))
        
        expduration = int(re.search("(?<=_u)\d+", fname).group(0))
        
        collectl_fname = [x for x in os.listdir(".") if "%s-server-0" % fname in x]
        
        if len(collectl_fname) > 0:
            df_stats = load_collectl(collectl_fname[0])
            newdf = pd.concat([df_tput, df_stats])
        else:
            newdf = df_tput
        
        newdf["expduration"] = expduration
        newdf["fname"] = fname  # for finding weird data
        newdf["ncores"] = int(fname.split("cores")[0])
        pds.append(newdf)

    return (basename, pd.concat(pds).apply(pd.to_numeric, errors='coerce'))

In [5]:
dfs = {}
os.listdir(".")

['2numa_u80_c2048_23.dat-server-0-20171218.tab',
 '2numa_u80_c2048_21.dat',
 '2numa_u80_c2048_26.dat',
 '3numa_u80_c2048_20.dat-server-0-20171218.tab',
 '3numa_u80_c2048_8.dat-server-0-20171218.tab',
 '3numa_u80_c2048_12.dat',
 '2numa_u80_c2048_28.dat',
 '4numa_u80_c2048_16.dat-server-0-20171218.tab',
 '2numa_u80_c2048_4.dat-server-0-20171218.tab',
 '3numa_u80_c2048_15.dat',
 '3numa_u80_c2048_20.dat',
 '3numa_u80_c2048_23.dat-server-0-20171218.tab',
 '4numa_u80_c2048_1.dat',
 '3numa_u80_c2048_8.dat',
 '3numa_u80_c2048_27.dat',
 '4numa_u80_c2048_6.dat',
 '2numa_u80_c2048_20.dat-server-0-20171218.tab',
 '2numa_u80_c2048_7.dat-server-0-20171218.tab',
 '3numa_u80_c2048_29.dat',
 '2numa_u80_c2048_3.dat',
 '2numa_u80_c2048_13.dat',
 '4numa_u80_c2048_8.dat',
 '3numa_u80_c2048_1.dat',
 '2numa.conf',
 '4numa_u80_c2048_29.dat-server-0-20171218.tab',
 '2numa_u80_c2048_4.dat',
 '2numa_u80_c2048_14.dat',
 '4numa_u80_c2048_15.dat-server-0-20171218.tab',
 '3numa_u80_c2048_6.dat',
 '4numa_u80_c2048_14

In [56]:
for exp in [
    "1numa",
    #"2numa",
    #"3numa",
    #"4numa"
]:
    print ("Working on experiment %s..." % exp)
    label, df = load_data_and_stats(exp)
    df = df.fillna(df.mean())
    
    # Remove rows where ncli != 2048
    dfs[exp] = df
    

Working on experiment 1numa...
trying to match...


OSError: [Errno 5] Input/output error: '.'

In [ ]:
grouped = pd.concat([df for key, df in dfs.items()]).groupby("numa")
agged = grouped.agg([np.std, np.mean])

In [ ]:
grouped.get_group(1)

In [ ]:
data = []

barwidth = 0.5

x = [numacount + " node(s)" for numacount in 
    agged.index.astype(int).astype(str)]

# Throughput
data.append(Bar(
    x = x,
    y = agged[("[CPU]Sys%", "mean")],
    yaxis = 'y2',
    name = 'CPU Sys%',
    width = barwidth,
    opacity=0.6
))
data.append(Bar(
    x = x,
    y = agged[("[CPU]Soft%", "mean")],
    yaxis = 'y2',
    name = 'CPU Soft%',
    width = barwidth,
    opacity=0.6
))
data.append(Bar(
    x = x,
    y = agged[("[CPU]User%", "mean")],
    yaxis = 'y2',
    name = 'CPU User%',
    width = barwidth,
    opacity=0.6
))

for label, df in grouped:
    data.append(Box(
        y = df["throughput"],
        name = "%s node(s)" % label,
        boxpoints = 'all',
        #boxmean = "sd",
        showlegend = False,
        marker = dict(color = "rgb(8, 81, 156)",
                     size = 3),
        jitter = 0.5,
    ))

    
layout = Layout (
    xaxis = {'title': 'number of NUMA nodes'},
    yaxis = {'title': 'throughput (pps)', 'rangemode': 'tozero'},
    yaxis2 = {'title': 'CPU utilization', 
              'side': 'right', 
              'overlaying': 'y',
              'showgrid': False},
    title = 'Small-packet throughput with varying NUMA count: 2048 clients',
    #legend = {'x': 0.01, 'y': 1},
    #boxmode = "group"
    barmode = "stack",
    boxgap = 0.5
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename='tcp-cpu-throughput-varynuma-2048cli')

In [32]:
ratios = []
ratios.append(100*agged[("[CPU]User%", "mean")]/agged[("[CPU]Totl%", "mean")])
ratios.append(100*agged[("[CPU]Soft%", "mean")]/agged[("[CPU]Totl%", "mean")])
ratios.append(100*agged[("[CPU]Sys%", "mean")]/agged[("[CPU]Totl%", "mean")])
ratios

[numa
 4    4.455216
 dtype: float64, numa
 4    27.948884
 dtype: float64, numa
 4    66.858389
 dtype: float64]

In [24]:
data = []

i = 1
for label, df in grouped:
    data.append(Histogram(x=df["throughput"],
                          nbinsx=10,
                          name="%s node(s)" % label,
                          opacity=0.6,
                          histnorm="probability"))

layout = Layout (
    xaxis = {'title': 'throughput (pps)'},
    yaxis = {'title': 'frequency'},
    title = 'Throughput frequency for varying core counts',
    showlegend = True,
) 
fig = Figure(data=data, layout=layout)
    
py.iplot(fig, filename="tput_frequency_durations_ncores")